# Audio Input on the fast ADC upload to Edge Impulse

This example shows how an audio signal can be captured using the fast input IN2.

Mercury is the Jupyter FPGA overlay for the Red Pitaya.

In [ ]:
from redpitaya.overlay.mercury import mercury as overlay
import IPython.display as ipd
import matplotlib.pyplot as plt

#Audio processing requirements
import numpy as np
from scipy.io.wavfile import write

#Edge Impulse requirements
import json
import time, hmac, hashlib
import requests
import re, socket
import math

fpga = overlay()

Make an instance of the oscilloscope object is used for the input. The first argument is the channel index in range [0,1]. The second argument is the voltage range in volts, should be either 1.0 (default if unspecified) or 20.0. This is set to channel 1 (IN2) and 1.0 Volts the Low setting on the jumper.

In [ ]:
in2 = fpga.osc(1, 1.0)

User should specify:
1. Sample rate decimation is an integer in range [1, 2\*\*17] = [1, 131072].
2. Number of samples stored before and after trigger, for a forced trigger only post trigger samples are enough.
3. Input level trigger source should (depending on the test purpose) be disabled (it is enabled by default)

30,000 samples per second is the minimum sample rate IPython.display will support. The minimum was chosen for this example to maximise the recording time using the oscilloscope object buffer.

In [ ]:
# data rate decimation 
in2.decimation = 41667 #125 Msps / 41667 = 30 Ksps

# trigger timing [sample periods]
N = in2.buffer_size
in2.trigger_pre  = 0
in2.trigger_post = N

# disable hardware trigger sources - the Jupyter notebook will trigger the start of audio recording
in2.trig_src = 0

The oscilloscope should be reset, started, and a software trigger should be issued.
A loop is checking if the desired number of data samples was already stored in the buffer.

The same number of samples as the sum of pre/post trigger delays should be read from the buffer.
The data read is aligned to the last sample, which will be always present as the last sample in the returned array.

In [ ]:
def record_audio():
    # synchronization and trigger sources are the default,
    # which is the module itself
    in2.reset()
    in2.start()
    in2.trigger()
    print ('Recording started')
    # wait for data
    while (in2.status_run()): pass
    print ('Recording complete')
    data = in2.data(N)
    return data

audio_data_returned = record_audio()

The audio data can then be displayed as an audio file that can be played or downloaded.

In [ ]:
def display_audio(audio_data,sr = 3000): #default sample rate 3 Ksps
    return ipd.Audio(audio_data, rate=sr) # load a NumPy array

python_display_audio = display_audio(audio_data_returned, sr = 3000)
python_display_audio

Once the recording has been captured it can be displayed as a waveform.

In [ ]:
def plot_audio(audio_data):
    # show the part of the buffer requested by pre/post trigger timing - in this example the whole buffer.
    plt.plot(audio_data_returned)
    plt.show()

plot_audio(audio_data_returned)

In [ ]:
def save_audio(audio_data, file, sr = 3000): #default sample rate 3 Ksps
    write("float_wave_test_raw.wav", sr, audio_data)

# Add the files you want to upload to Edge Impulse
files = ['float_wave_test_raw.wav']
file = files[0] #save the first one
save_audio(audio_data_returned, file, sr = 3000)

In [ ]:
# Install requests via: `pip3 install requests`
import requests
import os
from app_keys import * #Import personal keys for the Edge Impulse project

def upload_audio2edge_impulse(files, label = 'red-pitaya'): # Replace the label with your own.
    
    # Upload the file to Edge Impulse using the API, and print the response.
    res = requests.post(url='https://ingestion.edgeimpulse.com/api/training/files',
                        headers={
                            'x-label': label,
                            'x-api-key': api_key,
                        },
                        # Creating the data payload for the request.
                        files=(('data', (os.path.basename(i), open(
                            i, 'rb'), 'audio/wav')) for i in files)
                        )
    
    if (res.status_code == 200):
        print('Uploaded file(s) to Edge Impulse\n', res.status_code, res.content)
    else:
        print('Failed to upload file(s) to Edge Impulse\n',
              res.status_code, res.content)

upload_audio2edge_impulse(files)

## Put All The Functions Together
Record and upload a sound as part of the training data as the sound "Red Pitaya" with the <code>red-pitaya</code> label in [Edge Impulse](https://edgeimpulse.com/).

In [ ]:
audio_data_returned = record_audio()

python_display_audio = display_audio(audio_data_returned, sr = 3000)

plot_audio(audio_data_returned)

# Add the files you want to upload to Edge Impulse
files = ['float_wave_test_raw.wav']
file = files[0] #save the first one
save_audio(audio_data_returned, file, sr = 3000)

upload_audio2edge_impulse(files)
python_display_audio

## Minimal Version
This version doesn't include the displays, and uses a previously defined file name.

In [39]:
audio_data_returned = record_audio()
save_audio(audio_data_returned, file, sr = 3000)
upload_audio2edge_impulse(files)

Recording started
Recording complete
Uploaded file(s) to Edge Impulse
 200 b'{"success":true,"files":[{"success":true,"projectId":440448,"sampleId":1107089003,"fileName":"float_wave_test_raw.wav.53ph4ne2.json"}]}'
